In [1]:
import os
import time 
import pickle

import numpy as np
from copy import deepcopy

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_white"

In [2]:
# Root of Project
ROOT = '/home/dev/ws_percept/src'

# Cameras
IMAGE_SIZE  = 240
CAMERAS = ['cam1', 'cam2', 'cam3']

# Scene
# [x_min, y_min, z_min, x_max, y_max, z_max] - the metric volume to be voxelized
SCENE_BOUNDS    = (-1.5, -1.5, -1.5, 1.50, 1.5, 1.5)

# Agents
AGENTS = ['panda0', 'panda1']

SHOW_CPH_VIZ = False

In [3]:
# load data
DATASET_PATH = '../outputs/dualarms_3cam/dualarms_3cam_with_fk.pkl'
with open(DATASET_PATH, 'rb') as f:
    _obs = pickle.load(f)

## Setup PCD, Mesh

In [4]:
def constrain_scene_bounds(data):
    global SCENE_BOUNDS

    nX, nY, nZ, X, Y, Z = SCENE_BOUNDS

    X_mask = (data[:,0] > nX) & (data[:,0]<X)
    Y_mask = (data[:,1] > nY) & (data[:,1]<Y)
    Z_mask = (data[:,2] > nZ) & (data[:,2]<Z)
    mask = (X_mask)&(Y_mask)&(Z_mask) 

    return data[mask]

def get_pcd_and_rgb(camera_obs:dict):
    pcd = camera_obs['pointcloud'].reshape(-2,3)
    rgb = camera_obs['rgb'].reshape(-2,3)
    return pcd, rgb

def get_pcd(camera_obs:dict):
    pcd = camera_obs['pointcloud'].reshape(-2,3)
    return pcd


obs = _obs[-1]
camera_obs = obs[CAMERAS[0]]
pcd, rgb = get_pcd_and_rgb(camera_obs)

In [5]:
from scipy.spatial.transform import Rotation as R
import cupoch as cph

ROBOT_URDF_PATH = 'outputs/testdata/franka_panda_cupoch/panda.urdf'
URDF_JOINT_PREFIX = 'panda_joint'

path = os.path.join(ROOT, ROBOT_URDF_PATH)
kin = cph.kinematics.KinematicChain(path)


def get_tf_matrix(
    translation:np.array,
    rotation:np.array, 
) -> np.array:

    r = R.from_rotvec(rotation)
    r = r.as_matrix()

    tf = np.eye(4)
    tf[:3,:3] = r
    tf[:3,3] = translation

    return tf

def manually_offset_base(ref:np.array, translation_offset) -> np.array:
    offset = np.eye(4)
    offset[:3, 3] = translation_offset
    return np.dot(ref, offset)

def get_mesh_using_forward_kinematics(
    joints_positions:np.array,
    global_position:np.array,
    global_rotation:np.array
):
    global URDF_JOINT_PREFIX

    # create joint_map, used for fk on urdf model
    joint_map = {'%s%d' % (URDF_JOINT_PREFIX, i+1): val for i, val \
                in enumerate(joints_positions)} 

    # create transformation matrix
    tf_matrix = get_tf_matrix(global_position, global_rotation)

    # manual offset, to be removed 
    offset = np.array([0.04, 0.0, -0.068])
    tf_matrix = manually_offset_base(tf_matrix, offset)
    
    # do forward kinematics
    poses = kin.forward_kinematics(joint_map, tf_matrix)
    
    # store mesh geometries
    meshes = kin.get_transformed_visual_geometry_map(poses)
    
    return list(meshes.values())

[2024-08-27 17:15:40.281] [warning] Read OBJ failed: Material file [ link0.obj.mtl ] not found in a path : /home/dev/ws_percept/src/outputs/testdata/franka_panda_cupoch/meshes/collision/
Failed to load material file(s). Use default material.
material [ 'DefaultMaterial' ] not found in .mtl

[2024-08-27 17:15:40.652] [warning] Read OBJ failed: Material file [ link0.obj.mtl ] not found in a path : /home/dev/ws_percept/src/outputs/testdata/franka_panda_cupoch/meshes/collision/
Failed to load material file(s). Use default material.
material [ 'DefaultMaterial' ] not found in .mtl

[2024-08-27 17:15:40.653] [warning] Read OBJ failed: Material file [ link1.obj.mtl ] not found in a path : /home/dev/ws_percept/src/outputs/testdata/franka_panda_cupoch/meshes/collision/
Failed to load material file(s). Use default material.
material [ 'DefaultMaterial' ] not found in .mtl

[2024-08-27 17:15:40.681] [warning] Read OBJ failed: Material file [ link2.obj.mtl ] not found in a path : /home/dev/ws_perc

## PCR

In [6]:
def get_pcds_for_registration(obs):
    global CAMERAS

    pcds = list()
    for CAMERA in CAMERAS:
        camera_obs = obs[CAMERA]
        pcd, rgb = get_pcd_and_rgb(camera_obs)
        pcd = constrain_scene_bounds(pcd)
        pcds.append(pcd)

    return pcds

pcds = get_pcds_for_registration(obs)

In [7]:
def do_point_cloud_registration(pcds):
    assert len(pcds) > 1
    def register(
        source_gpu:cph.geometry.PointCloud, 
        target_gpu:cph.geometry.PointCloud,
    ) -> cph.geometry.PointCloud:

        threshold = 0.02 # what does this do?
        trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0],
                                [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])

        # register pointclouds
        start = time.time()
        reg_p2p = cph.registration.registration_icp(
            source_gpu,
            target_gpu,
            threshold,
            trans_init.astype(np.float32),
            cph.registration.TransformationEstimationPointToPlane(),
        )

        source_gpu.transform(reg_p2p.transformation)

        # remove outliers
        NEIGHBOURS = 2
        source_gpu, ind = source_gpu.remove_statistical_outlier(nb_neighbors=NEIGHBOURS, std_ratio=2.0)
        target_gpu, ind = target_gpu.remove_statistical_outlier(nb_neighbors=NEIGHBOURS, std_ratio=2.0)


        elapsed_time = time.time() - start
        print("ICP (GPU) [sec]:", elapsed_time) # adding outlier removal adds ~25ms
    
        return source_gpu+target_gpu


    # load first and second pcd
    source_gpu = cph.geometry.PointCloud(
        cph.utility.HostVector3fVector(pcds[0])
    )
    target_gpu = cph.geometry.PointCloud(
        cph.utility.HostVector3fVector(pcds[1])
    )
    source_gpu = register(source_gpu, target_gpu)

    # merge all other pcds with new source
    for pcd in pcds[2:]:
        target_gpu = cph.geometry.PointCloud(
            cph.utility.HostVector3fVector(pcd)
        )
        source_gpu = register(source_gpu, target_gpu)


    return source_gpu


pcd = do_point_cloud_registration(pcds)

[2024-08-27 17:15:41.126] [error] TransformationEstimationPointToPlane and TransformationEstimationColoredICP require pre-computed target normal vectors.
ICP (GPU) [sec]: 0.03101515769958496
[2024-08-27 17:15:41.244] [error] TransformationEstimationPointToPlane and TransformationEstimationColoredICP require pre-computed target normal vectors.
ICP (GPU) [sec]: 0.029599428176879883


## RBS

In [8]:
def get_bb_from_mesh(
    mesh:cph.geometry.TriangleMesh
) -> cph.geometry.AxisAlignedBoundingBox:

    AABB = mesh.get_axis_aligned_bounding_box()
    if AABB.is_empty():
        print('empty mesh detected')
        return None       
    else:
        return AABB
    

def perform_rbs_on_pointcloud_using_bb(obs, pcd):

    n = len(pcd.points)
    masks = list()
    meshes_list = list()
    for agent in AGENTS:
        print('\nAgent: %s' % agent)
        agent_obs = obs[agent]
        
        # get all meshes of the agent using fk
        meshes = get_mesh_using_forward_kinematics(
            agent_obs['joint_pos'],
            agent_obs['global_pos'],
            agent_obs['global_ang'],
        )
        meshes_list += meshes

        # find points within BBs and create masks
        start = time.time()
        for mesh in meshes:
            bb = get_bb_from_mesh(mesh)

            if bb is not None:
                mask = np.zeros(n, dtype=bool)

                indices_points_within_bb = np.asarray(
                    bb.get_point_indices_within_bounding_box(
                        pcd.points
                    ).cpu()
                )
                if len(indices_points_within_bb)>0:
                    mask[indices_points_within_bb] = True
                    masks.append(mask)

        elapsed_time = time.time() - start
        print("RBS (CPU+GPU) [sec]:", elapsed_time)


    mask = np.column_stack(tuple(masks)).any(axis=1)
    mask = ~mask # inverting mask to get pcd which were not within any BBs
    
    pcd_arr = np.asarray(pcd.points.cpu())
    NEIGHBOURS = 10
    pcd = cph.geometry.PointCloud(cph.utility.HostVector3fVector(pcd_arr[mask]))
    pcd = pcd.remove_statistical_outlier(nb_neighbors=NEIGHBOURS, std_ratio=10.0)
    return  pcd[0], meshes_list

rbs_pcd, meshes = perform_rbs_on_pointcloud_using_bb(obs, pcd)


Agent: panda0
empty mesh detected
empty mesh detected
RBS (CPU+GPU) [sec]: 0.01578044891357422

Agent: panda1
empty mesh detected
empty mesh detected
RBS (CPU+GPU) [sec]: 0.012721776962280273


In [9]:
if SHOW_CPH_VIZ:
    cph.visualization.draw_geometries([rbs_pcd]+meshes)

## Voxelization

In [10]:
def convert_pointcloud_to_voxels(pcd):
    cubic_size = 2.0
    # voxel_resolution = 100.0
    voxel_resolution = 30.0


    voxel_size = cubic_size / voxel_resolution

    # create voxel grid
    start = time.time()
    voxels = cph.geometry.VoxelGrid.create_from_point_cloud_within_bounds(
        pcd,
        voxel_size=cubic_size / voxel_resolution,
        min_bound=(-cubic_size / 2, -cubic_size / 2, -cubic_size / 2),
        max_bound=(cubic_size / 2, cubic_size / 2, cubic_size / 2),
    )
    elapsed_time = time.time() - start
    print("Voxelization (GPU) [sec]:", elapsed_time) # adding outlier removal adds ~25ms


    return voxels, voxel_size

voxels, voxel_size = convert_pointcloud_to_voxels(rbs_pcd)

Voxelization (GPU) [sec]: 0.008053779602050781


In [11]:
if SHOW_CPH_VIZ:
    cph.visualization.draw_geometries([voxels])

In [57]:
radius = 0.06
VIZ = True

if VIZ:
    def voxel2sphere(voxels):
        global voxel_size, radius
        v = voxels.voxels.cpu()
        offset = voxels.get_min_bound()

        positions = np.array(list(v.keys()))

        positions = (positions - np.array([min(positions[:,0]), min(positions[:,1]), min(positions[:,2])]))
        positions = positions*voxel_size
        positions += (offset + voxel_size/2)

        spheres = cph.geometry.TriangleMesh.create_sphere(radius=0.01, resolution=5).translate(positions[0])
        for pos in positions[1:]:
            spheres += cph.geometry.TriangleMesh.create_sphere(
                radius=radius, 
                resolution=5).translate(pos)
        return spheres, positions

    spheres, positions  = voxel2sphere(voxels)    

    def meshColorscale(mesh):
        vertices = np.asarray(mesh.vertices.cpu())
        z_values = vertices[:, 2]

        # Normalize the z values to range [0, 1]
        z_min = z_values.min()
        z_max = z_values.max()
        z_norm = (z_values - z_min) / (z_max - z_min)

        # Create a color map (using a matplotlib colormap, e.g., 'viridis')
        cmap = plt.get_cmap('turbo')
        colors = cmap(z_norm)[:, :3]  # Get RGB values

        # Apply the colors to the mesh
        mesh.vertex_colors = cph.utility.Vector3fVector(colors)
        return mesh

    spheres = meshColorscale(spheres)

else:
    def voxel2sphere(voxels):
        global voxel_size
        start = time.time()
        v = voxels.voxels.cpu()
        offset = voxels.get_min_bound()

        positions = np.array(list(v.keys()))

        positions = (positions - np.array([min(positions[:,0]), min(positions[:,1]), min(positions[:,2])]))
        positions = positions*voxel_size
        positions += (offset + voxel_size/2)

        elapsed_time = time.time() - start
        print("Voxel2Sphere (CPU) [sec]:", elapsed_time) # adding outlier removal adds ~25ms

        return positions

    positions  = voxel2sphere(voxels)    

p = np.array(pcd.points.cpu())
print(min(p[:,0]),min(p[:,1]),min(p[:,2]))
print(max(p[:,0]),max(p[:,1]),max(p[:,2]))

print(min(positions[:,0]),min(positions[:,1]),min(positions[:,2]))
print(max(positions[:,0]),max(positions[:,1]),max(positions[:,2]))

-1.4999754 -1.4988322 -1.555519e-05
1.4987465 1.4999558 1.4999748
-1.5 -1.5 -0.03333326801657677
1.5 1.5 1.5000000653167564


In [58]:
if VIZ:
    cph.visualization.draw_geometries([spheres])

## Moving Workspace

In [14]:
# # visualize workspace
# def create_cube_edges(pos_, detect_shell_rad_ ):
#     # Define the 8 vertices of the cube
#     vertices = np.array([
#         [0, 0, 0],  # vertex 0
#         [1, 0, 0],  # vertex 1
#         [1, 1, 0],  # vertex 2
#         [0, 1, 0],  # vertex 3
#         [0, 0, 1],  # vertex 4
#         [1, 0, 1],  # vertex 5
#         [1, 1, 1],  # vertex 6
#         [0, 1, 1]   # vertex 7
#     ])

#     # Define the edges by connecting the vertices
#     edges = np.array([
#         [0, 1], [1, 2], [2, 3], [3, 0],  # bottom face
#         [4, 5], [5, 6], [6, 7], [7, 4],  # top face
#         [0, 4], [1, 5], [2, 6], [3, 7]   # vertical edges
#     ])

#     vertices = detect_shell_rad_*(vertices - np.array([0.5,0.5,0.5])) + pos_

#     # Create a LineSet to represent the edges of the cube
#     line_set = cph.geometry.LineSet()
#     line_set.points = cph.utility.Vector3fVector(vertices)
#     line_set.lines = cph.utility.Vector2iVector(edges)

#     return line_set

# agent_pos = np.array([-0.5,0.0,0.75])
# detect_shell_rad_ = 0.5
# cph.visualization.draw_geometries([
#     voxels, 
#     create_cube_edges(agent_pos, detect_shell_rad_)
# ])

In [15]:
import torch, math
from torch import tensor
import torchvision as tv
import torchvision.transforms.functional as tvf
from torchvision import io
from torch.utils.cpp_extension import load_inline

In [16]:
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [17]:
%load_ext wurlitzer

In [18]:
def load_cuda(cuda_src, cpp_src, funcs, opt=False, verbose=False, build_name="inline_ext"):
    return load_inline(
        cuda_sources=[cuda_src], cpp_sources=[cpp_src], functions=funcs,
        extra_cuda_cflags=["-O2"] if opt else [], verbose=verbose, name=build_name)

In [35]:
cuda_src = r'''
#include <torch/extension.h>
#include <stdio.h>
#include <c10/cuda/CUDAException.h>

#define CHECK_CUDA(x) TORCH_CHECK(x.device().is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)


__global__ void radial_search_kernel(
    const float* points, 
    const float* target, 
    float obstacle_rad,
    float detect_shell_rad, 
    int num_points, 
    int* output, 
    int* output_count
){
    int idx = blockIdx.x*blockDim.x + threadIdx.x;
    if (idx >= num_points) return;
    
    // get dx, dy, dz
    float dx = points[idx * 3] - target[0];
    float dy = points[idx * 3 + 1] - target[1];
    float dz = points[idx * 3 + 2] - target[2];

    float distance = sqrtf(dx * dx + dy * dy + dz * dz) - obstacle_rad;
    if (distance <= detect_shell_rad) {
        int insert_idx = atomicAdd(output_count, 1);
        output[insert_idx] = idx;
    }

}

torch::Tensor radial_search_cuda(
    torch::Tensor points, 
    torch::Tensor target, 
    float obstacle_rad, 
    float detect_shell_rad, 
    int num_points
) {
    CHECK_INPUT(points);

    auto output = torch::zeros({num_points}, torch::dtype(torch::kInt32).device(torch::kCUDA));
    auto output_count = torch::zeros({1}, torch::dtype(torch::kInt32).device(torch::kCUDA));

    int threads = 256;
    int blocks = (num_points + threads - 1) / threads;        
    radial_search_kernel<<<blocks, threads>>>(
        points.data_ptr<float>(),
        target.data_ptr<float>(),
        obstacle_rad, 
        detect_shell_rad,
        num_points,
        output.data_ptr<int>(),
        output_count.data_ptr<int>()
    );
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    cudaDeviceSynchronize();

    int result_size = output_count.item<int>();
    return output.slice(0, 0, result_size);
}
'''
cpp_src = "torch::Tensor radial_search_cuda(torch::Tensor points, torch::Tensor target, float obstacle_rad, float detect_shell_rad, int num_points);"
module = load_cuda(cuda_src, cpp_src, ['radial_search_cuda'], verbose=True)

Using /home/dev/.cache/torch_extensions/py310_cu124 as PyTorch extensions root...
The input conditions for extension module inline_ext have changed. Bumping to version 2 and re-building as inline_ext_v2...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/dev/.cache/torch_extensions/py310_cu124/inline_ext/build.ninja...
/home/dev/miniconda3/envs/percept/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module inline_ext_v2...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module inline_ext_v2...


In [109]:
%%time
points = torch.from_numpy(positions).contiguous().cuda().flatten().float()
target = torch.from_numpy(np.array([0.0,0.0,1.0])).contiguous().cuda().flatten().float()
res = module.radial_search_cuda(
    points, 
    target, 
    0.06,
    1.0,
    positions.shape[0]
).cpu()
print(res.shape)

torch.Size([1094])
CPU times: user 680 μs, sys: 358 μs, total: 1.04 ms
Wall time: 777 μs


In [100]:
res

tensor([1798, 1820, 1925,  ..., 3158, 3160, 3969], dtype=torch.int32)

In [112]:
positions.shape

(3971, 3)

In [110]:
def create_spheres(positions, radius):
    spheres = cph.geometry.TriangleMesh.create_sphere(
        radius=radius, 
        resolution=5).translate(positions[0])
    for pos in positions[1:]:
        spheres += cph.geometry.TriangleMesh.create_sphere(
            radius=radius, 
            resolution=5).translate(pos)
    return spheres

spheres = create_spheres(positions[res.numpy()], radius)

def meshColorscale(mesh):
    vertices = np.asarray(mesh.vertices.cpu())
    z_values = vertices[:, 2]

    # Normalize the z values to range [0, 1]
    z_min = z_values.min()
    z_max = z_values.max()
    z_norm = (z_values - z_min) / (z_max - z_min)

    # Create a color map (using a matplotlib colormap, e.g., 'viridis')
    cmap = plt.get_cmap('turbo')
    colors = cmap(z_norm)[:, :3]  # Get RGB values

    # Apply the colors to the mesh
    mesh.vertex_colors = cph.utility.Vector3fVector(colors)
    return mesh

spheres = meshColorscale(spheres)

cph.visualization.draw_geometries([spheres])